In [11]:
%%capture
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #usunięcie warningów tensorflow
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install intel-tensorflow
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install sklearn

In [12]:
import pandas as pd
from keras import optimizers
from keras.layers import Dense
from keras.models import Sequential
from sklearn.model_selection import ShuffleSplit
import sklearn as skl
import keras.initializers

Odczytujemy dane cen domów. Usuwamy kolumnę z id, i oddzielamy cenę od pozostałych kolumn

In [13]:
def get_data():
    read = pd.read_csv('kc_house_data.csv')
    prices, other_data = read["price"], read[[*read.columns[3:], 'date']]
    return prices, other_data

Przygotowanie danych. Z daty wyciągamy rok i miesiąc, i przygotowujemy kolumny z wartościami binarnymi
mówiące w którym okresie dom został sprzedany.
Dodajemy też kolumny określające binarnie lokalizację budynku na podstawie kodu pocztowego.
Następnie skalujemy dane.

In [14]:
def prepare_data(d):
    d['floors'] = d['floors'].astype('double')
    for s in ['2014', '2015']:
        d['y_' + s] = [1 if y[0:4] == s else 0 for y in d['date']]
    for i in range(1, 13):
        s = str(i) if len(str(i)) == 2 else '0' + str(i)
        d['m_' + s] = [1 if y[4:6] == s else 0 for y in d['date']]
    d = d[[c for c in d.columns if c != 'date']]
    zipcodes = set(d['zipcode'])
    for z in zipcodes:
        d['z_' + str(z)] = [1 if e == z else 0 for e in d['zipcode']]
    d = d[[c for c in d.columns if c != 'zipcode']]
    return skl.preprocessing.minmax_scale(d)

Przygotowujemy sieć neuronową składającą się z 5 warstw.

In [15]:
def get_model():
    model = Sequential()
    length = 101
    activ = keras.activations.relu
    model.add(Dense(35, activation=activ, input_shape=(length,)))
    model.add(Dense(35, activation=activ))
    model.add(Dense(35, activation=activ))
    model.add(Dense(35, activation=activ))
    model.add(Dense(1, activation=activ))
    model.compile(loss='mae', optimizer=optimizers.adam(0.002), metrics=[keras.metrics.mae, keras.metrics.mse])
    return model

Ceny dla celów predykcji będą transformowane, więc przygotowujemy funkcję obliczająca błąd która
odwraca transformację cen. Jako funkcję błędu wykorzystujemy średni błąd bezwzględny i średni błąd procentowy.

In [26]:
def error_func(sc, y, pred):
    exp_res = sc.inverse_transform(y)
    res = sc.inverse_transform(pred)
    mae = skl.metrics.mean_absolute_error(exp_res, res)
    perc = abs(exp_res - res)/exp_res
    avg_perc = sum(perc)/len(perc)
    return mae, avg_perc[0]


Skalujemy ceny, tak by były one nie większe niż 1.
Następnie trenujemy sieci neuronowe, i za pomocą walidacji krzyżowej sprawdzamy błąd klasyfikatora.

In [27]:
def main_nn():
    epochs = 1000
    (prices, data) = get_data()
    prepared = prepare_data(data)
    prices = prices.values.reshape(-1, 1)
    sc = skl.preprocessing.MinMaxScaler()
    sc.fit(prices)
    ss = ShuffleSplit(n_splits=5, test_size=0.2, random_state=24)
    splitted = ss.split(prepared, prices)
    transformed_prices = sc.transform(prices)
    maes = []
    perc = []
    for i, (train_index, val_index) in enumerate(splitted):
        model = get_model()
        X_train, X_val = prepared[train_index, :], prepared[val_index, :]
        y_train, y_val = transformed_prices[train_index], transformed_prices[val_index]
        model.fit(X_train, y_train, epochs=epochs, batch_size=len(X_train), verbose=0)
        mae, avg_perc = error_func(sc, y_val, model.predict(X_val))
        maes.append(mae)
        perc.append(avg_perc)
        print("MAE " + str(i+1) + " : " + str(mae) + "Avg perc error: " + str(avg_perc))
    print("Avg MAE: " + str(sum(maes)/len(maes)) + " avg percentage erro: " + str(sum(perc)/len(perc)))

In [28]:
main_nn()

MAE 1 : 78424.6945805864Avg perc error: 0.14322472144463058
MAE 2 : 77447.83255262549Avg perc error: 0.14408411764006937
MAE 3 : 80003.89282796958Avg perc error: 0.14868674225637693
MAE 4 : 80541.28988838769Avg perc error: 0.1466532575016918
MAE 5 : 71942.10447027527Avg perc error: 0.14101258219152243
Avg MAE: 77671.96286396889 avg percentage erro: 0.1447322842068582


Średni błąd bezwzględny we wszystkich moich próbach wynosił od 75000 do 80000, a średni błąd precentowy od 12% do 15%